In [ ]:
import glob
import json
import importlib
import inspect
import os
from subprocess import check_call

In [ ]:
default_convert_format = ['script', 'markdown', 'html']

In [ ]:
c = get_config()

c.NotebookApp.allow_origin = '*'
c.ExecutePreprocessor.timeout = None

In [ ]:
def post_save(model, os_path, contents_manager):
    """Automatically convert notebooks on save if files exist 
    in the working directory.
    
    Convert all notebooks to python scripts if there are 
    no spaces or `-` in the filename.
    
    %%bash
    touch _script.py 
    
    Suggested convention: 
    use double underscore to indicate another nbconvert format  ie. __html.py.
    """
    # Only convert notebooks
    if model['type'] != 'notebook': return

    directory, fname = os.path.split(os_path)

    call = ['jupyter', 'nbconvert']

    for fmt in default_convert_format:
        config_file = os.path.join(directory, '_{}.py'.format(fmt))
        if os.path.isfile(config_file):
            # Only convert valid names to python.
            if fmt == 'script' and (' ' in fname or '-' in fname):
                continue
            call.extend(['--to', fmt])
            call.extend(['--config', config_file])
            call.append(fname)
            check_call(call, cwd=directory)

In [ ]:
c.FileContentsManager.post_save_hook = post_save